In [45]:
import numpy as np
import cv2

# import PIL.Image as Image
import os

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import glob

In [46]:
IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4", input_shape=IMAGE_SHAPE+(3,))
])

In [47]:
train_images_dict = {
    'combat': list(glob.glob('dataset_eyrc/Combat/*jpeg')),
    'destroyedbuidings': list(glob.glob('dataset_eyrc/DestroyedBuildings/*jpeg')),
    'fire': list(glob.glob('dataset_eyrc/Fire/*jpeg')),
    'humaniatrianaid': list(glob.glob('dataset_eyrc/Humanitarian Aid and rehabilitation/*jpeg')),
    'military': list(glob.glob('dataset_eyrc/Military vehicles and weapons/*jpeg')),
}
test_images_dict = {
    'combat': list(glob.glob('test_dataset_eyrc/Combat/*jpeg')),
    'destroyedbuidings': list(glob.glob('test_dataset_eyrc/DestroyedBuildings/*jpeg')),
    'fire': list(glob.glob('test_dataset_eyrc/Fire/*jpeg')),
    'humaniatrianaid': list(glob.glob('test_dataset_eyrc/Humanitarian Aid and rehabilitation/*jpeg')),
    'military': list(glob.glob('test_dataset_eyrc/Military vehicles and weapons/*jpeg')),
}



In [48]:
labels_dict = {
    'combat': 0,
    'destroyedbuidings': 1,
    'fire': 2,
    'humaniatrianaid': 3,
    'military': 4,
}

In [54]:
x_test, y_test,x_train,y_train = [], [],[],[]

for name, images in train_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(224,224))
        x_train.append(resized_img)
        y_train.append(labels_dict[name])
for name, images in test_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(224,224))
        x_test.append(resized_img)
        y_test.append(labels_dict[name])



In [55]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)


In [51]:
X_train_scaled = x_train / 255
X_test_scaled = x_test / 255


In [74]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [75]:
number_of_classes = 5

model = tf.keras.Sequential([
  pretrained_model_without_top_layer,
  tf.keras.layers.Dense(number_of_classes)
])

model.summary()

Model: "sequential_41"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_8 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dense_37 (Dense)            (None, 5)                 6405      
                                                                 
Total params: 2264389 (8.64 MB)
Trainable params: 6405 (25.02 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [76]:
model.compile(
  optimizer="adam",
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

model.fit(X_train_scaled, y_train, epochs=5)


Epoch 1/5
13/13 [==============================] - 7s 316ms/step - loss: 1.1218 - acc: 0.5542
Epoch 2/5
13/13 [==============================] - 4s 313ms/step - loss: 0.3619 - acc: 0.9395
Epoch 3/5
13/13 [==============================] - 4s 327ms/step - loss: 0.1845 - acc: 0.9748
Epoch 4/5
13/13 [==============================] - 4s 340ms/step - loss: 0.1244 - acc: 0.9773
Epoch 5/5
13/13 [==============================] - 4s 325ms/step - loss: 0.0929 - acc: 0.9849


In [77]:
model.evaluate(X_test_scaled,y_test)

1/1 [==============================] - 0s 485ms/step - loss: 0.0694 - acc: 1.0000


[0.069444939494133, 1.0]

In [78]:
model.save_weights('my_model_weights.h5')


In [79]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model

# Define the path to the image

image_path = 'dataset_eyrc/DestroyedBuildings/1.jpeg'

# Load your model
# loaded_model = load_model('my_model_weights.h5')
# image_path = glob.glob('dataset_eyrc/DestroyedBuildings/1.jpeg')
# Load and preprocess the image
image = cv2.imread(image_path)
# Resize the image to match the expected input shape of your model
image = cv2.resize(image, (224, 224))
image = image/255
# You may need to further preprocess the image according to your model's requirements.

# Make predictions
predictions = model.predict(np.expand_dims(image, axis=0))


# Find the predicted class
predicted_classes = np.argmax(predictions, axis=1)

# Define a class mapping
class_mapping = {0: 'combat', 1: 'destroyedbuilding', 2: 'Fire', 3: 'humanitarianaid', 4: 'militaryvehicles'}

# Map the predicted class to its corresponding label
event = [class_mapping[prediction] for prediction in predicted_classes]

# Print the result
print(event)


1/1 [==============================] - 0s 386ms/step
['destroyedbuilding']
